<a href="https://colab.research.google.com/github/priyanshu2103/Sanskrit-Hindi-Machine-Translation/blob/main/Unsupervised_MT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**```Unsupervised Machine Translation```**

This notebook runs unsupervised MT using cross-lingual embeddings and Moses

NOTE: This notebook requires GPU. RAM is a constraint, so larger data can lead to crashes.

In [ ]:
# Cloning our repo which contains the required data
!git clone https://github.com/priyanshu2103/Sanskrit-Hindi-Machine-Translation.git

Cloning into 'Sanskrit-Hindi-Machine-Translation'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 32 (delta 6), reused 27 (delta 4), pack-reused 0
Unpacking objects: 100% (32/32), done.


In [ ]:
# Downloading Hindi monolingual data provided by CFILT, IIT-B 

# !wget http://www.cfilt.iitb.ac.in/iitb_parallel/iitb_corpus_download/monolingual.hi.tgz    #works very slow generally, so can use the google drive id given below
!gdown --id 1lK1R454QHPwLbeHYtn7NQ9kDCLdzX00V
!rm -rf sample_data

Downloading...
From: https://drive.google.com/uc?id=1lK1R454QHPwLbeHYtn7NQ9kDCLdzX00V
To: /content/monolingual.hi.tgz
2.45GB [00:31, 76.8MB/s]


In [ ]:
!tar -xvzf monolingual.hi.tgz

monolingual/
monolingual/monolingual.hi


In [ ]:
!rm -rf /content/monolingual.hi.tgz

In [ ]:
%cd monolingual/

/content/monolingual


In [ ]:
!split -C 1024m --numeric-suffixes monolingual.hi monolingual_hindi_

In [ ]:
# downloading the Sanskrit Monolingual Dataset
%cd /content/
!gdown --id 1_qclc7unNLvToiDK8t2scIgj5oxJDEGm
!unzip Sanskrit_Monolingual_Dataset.zip
!rm -rf Sanskrit_Monolingual_Dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: Sanskrit_Monolingual_Dataset/बृहत्संहिता_अध्यायः_५७.txt  
  inflating: Sanskrit_Monolingual_Dataset/बृहत्संहिता_अध्यायः_५८.txt  
  inflating: Sanskrit_Monolingual_Dataset/बृहत्संहिता_अध्यायः_५९.txt  
  inflating: Sanskrit_Monolingual_Dataset/पञ्चविंशब्राह्मणम्_अध्यायः_८.txt  
  inflating: Sanskrit_Monolingual_Dataset/पञ्चविंशब्राह्मणम्_अध्यायः_९.txt  
  inflating: Sanskrit_Monolingual_Dataset/पञ्चसायकः.txt  
  inflating: Sanskrit_Monolingual_Dataset/पञ्चीकरणम्.txt  
  inflating: Sanskrit_Monolingual_Dataset/पञ्चीकरणवार्तिकम्.txt  
  inflating: Sanskrit_Monolingual_Dataset/पतञ्जलिचरितम्.txt  
  inflating: Sanskrit_Monolingual_Dataset/पतञ्जलिचरितम्_अष्टम:_सर्गः.txt  
  inflating: Sanskrit_Monolingual_Dataset/पतञ्जलिचरितम्_चतुर्थः_सर्गः.txt  
  inflating: Sanskrit_Monolingual_Dataset/पतञ्जलिचरितम्_द्वितीयः_सर्गः.txt  
  inflating: Sanskrit_Monolingual_Dataset/पतञ्जलिचरितम्_तृतीयः_सर्गः.txt  
  inflating: Sanskrit_Monolingual_

In [ ]:
# Concatenating the files into one single file
import os

text = ''
count = 0

for file in os.listdir("/content/Sanskrit_Monolingual_Dataset/"):
  count += 1
  # if count > 5000:
  #   break
  with open(os.path.join("/content/Sanskrit_Monolingual_Dataset/", file), "r") as f:
    text += f.read()

with open("/content/Sanskrit_Train_Data.txt", "w") as f:
  f.write(text)

In [ ]:
%cd /content/

/content


In [ ]:
%cd monolingual/
text = ''
with open("./monolingual_hindi_00", "r") as f:
    text_list = [next(f) for x in range(4270974)] # number of sentences in sanskrit monolingual data, if using any other data, please change this value here and in run.sh file in PBSMT folder later

for line in text_list:
  text += line

%cd ..
with open("/content/Hindi_Train_Data.txt", "w") as f:
  f.write(text) 

/content/monolingual
/content


In [ ]:
!wc Sanskrit_Train_Data.txt
!wc Hindi_Train_Data.txt
!mv Sanskrit_Train_Data.txt all.sa
!mv Hindi_Train_Data.txt all.hi
!cp /content/Sanskrit-Hindi-Machine-Translation/parallel-corpus/Ramcharitmanas/sanskrit_parallel_aligned.sa ./sanskrit_test_parallel.sa
!cp /content/Sanskrit-Hindi-Machine-Translation/parallel-corpus/Ramcharitmanas/hindi_parallel_aligned.hi ./hindi_test_parallel.hi

  4270974  32582159 713283822 Sanskrit_Train_Data.txt
  4270974  72825951 909856650 Hindi_Train_Data.txt


In [ ]:
!rm -rf monolingual
!rm -rf /content/Sanskrit_Monolingual_Dataset

Val and Test data

In [ ]:
%cd /content/
with open("./sanskrit_test_parallel.sa", "r") as f:
    text_list_sa = f.readlines()
with open("./hindi_test_parallel.hi", "r") as f:
    text_list_hi = f.readlines()

len1 = len(text_list_sa)
len2 = len(text_list_hi)
print(len1)
print(len2)

/content
5962
5962


In [ ]:
with open("./sanskrit_val_parallel.sa", "w") as f:
    for line in text_list_sa[:len1//2]:
      f.write(line)

with open("./hindi_val_parallel.hi", "w") as f:
    for line in text_list_hi[:len2//2]:
      f.write(line)

with open("./sanskrit_test_parallel.sa", "w") as f:
    for line in text_list_sa[len1//2:]:
      f.write(line)

with open("./hindi_test_parallel.hi", "w") as f:
    for line in text_list_hi[len2//2:]:
      f.write(line)

In [ ]:
%cd /content

/content


In [ ]:
# indic nlp library for tokenizing sanskrit and hindi
!pip install indic-nlp-library
!cp /content/Sanskrit-Hindi-Machine-Translation/indic_tokenize.py /usr/local/lib/python3.6/dist-packages/indicnlp/tokenize/indic_tokenize.py

In [ ]:
# This repo contains the code for unsupervised MT
!git clone https://github.com/priyanshu2103/UnsupervisedMT.git

Cloning into 'UnsupervisedMT'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 134 (delta 10), reused 21 (delta 8), pack-reused 111
Receiving objects: 100% (134/134), 287.85 KiB | 23.99 MiB/s, done.
Resolving deltas: 100% (45/45), done.


In [ ]:
# Downloading Moses binaries directly as installing from source is complicated
%cd /content
!wget http://www.statmt.org/moses/RELEASE-4.0/binaries/ubuntu-17.04.tgz
!tar -xvzf ubuntu-17.04.tgz
!rm -rf ubuntu-17.04.tgz

/content
--2020-10-30 12:26:55--  http://www.statmt.org/moses/RELEASE-4.0/binaries/ubuntu-17.04.tgz
Resolving www.statmt.org (www.statmt.org)... 129.215.197.184
Connecting to www.statmt.org (www.statmt.org)|129.215.197.184|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120382471 (115M) [application/x-gzip]
Saving to: ‘ubuntu-17.04.tgz’

ubuntu-17.04.tgz    100%[===================>] 114.81M  2.35MB/s    in 49s     

2020-10-30 12:27:45 (2.35 MB/s) - ‘ubuntu-17.04.tgz’ saved [120382471/120382471]

ubuntu-17.04/
ubuntu-17.04/training-tools/
ubuntu-17.04/training-tools/mkcls
ubuntu-17.04/training-tools/snt2cooc
ubuntu-17.04/training-tools/merge_alignment.py
ubuntu-17.04/training-tools/mgiza
ubuntu-17.04/moses/
ubuntu-17.04/moses/scripts/
ubuntu-17.04/moses/scripts/analysis/
ubuntu-17.04/moses/scripts/analysis/sg2dot.perl
ubuntu-17.04/moses/scripts/analysis/smtgui/
ubuntu-17.04/moses/scripts/analysis/smtgui/newsmtgui.cgi
ubuntu-17.04/moses/scripts/analysis/smtgui/

We use pre-trained cross-lingual embeddings provided by Kumar et. al, IIT-B

In [ ]:
%cd /content/
!wget http://www.cfilt.iitb.ac.in/~diptesh/embeddings/crosslingual/hisa.zip
!wget http://www.cfilt.iitb.ac.in/~diptesh/embeddings/crosslingual/sahi.zip
!unzip hisa.zip
!unzip sahi.zip
!rm -rf hisa.zip
!rm -rf sahi.zip

/content
--2020-10-30 12:27:49--  http://www.cfilt.iitb.ac.in/~diptesh/embeddings/crosslingual/hisa.zip
Resolving www.cfilt.iitb.ac.in (www.cfilt.iitb.ac.in)... 103.21.127.130
Connecting to www.cfilt.iitb.ac.in (www.cfilt.iitb.ac.in)|103.21.127.130|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 218127402 (208M) [application/zip]
Saving to: ‘hisa.zip’

hisa.zip            100%[===================>] 208.02M  9.70MB/s    in 59s     

2020-10-30 12:28:48 (3.55 MB/s) - ‘hisa.zip’ saved [218127402/218127402]

--2020-10-30 12:28:48--  http://www.cfilt.iitb.ac.in/~diptesh/embeddings/crosslingual/sahi.zip
Resolving www.cfilt.iitb.ac.in (www.cfilt.iitb.ac.in)... 103.21.127.130
Connecting to www.cfilt.iitb.ac.in (www.cfilt.iitb.ac.in)|103.21.127.130|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 218013472 (208M) [application/zip]
Saving to: ‘sahi.zip’

sahi.zip            100%[===================>] 207.91M  9.79MB/s    in 59s     

2020-10-30 12

In [ ]:
# clearing unrequired memory
del text_list_hi,text_list_sa

In [ ]:
# to train afresh, not read previously trained model present in the folder
# !rm -rf /content/UnsupervisedMT/PBSMT/MUSE/
# !rm -rf /content/UnsupervisedMT/PBSMT/data/
# !rm -rf /content/UnsupervisedMT/PBSMT/moses_train_sa-hi/

In [ ]:
# Disk space is also an issue, so clear un-necessary files
!rm -rf /content/Sanskrit-Hindi-Machine-Translation/
!rm -rf /content/UnsupervisedMT/NMT/
!rm -rf /content/hisa/
!rm -rf /content/UnsupervisedMT/*.md
!rm -rf /content/UnsupervisedMT/LICENSE
!rm -rf /content/UnsupervisedMT/illustration.png

In [1]:
%cd /content/UnsupervisedMT/PBSMT
!./run.sh

Streaming output truncated to the last 5000 lines.
BEST TRANSLATION: देव सगुन|UNK|UNK|UNK निर्गुन|UNK|UNK|UNK स्मृति पुरस्कार अनूप|UNK|UNK|UNK भूप|UNK|UNK|UNK सिरोमने|UNK|UNK|UNK इनका दसकंधरादि|UNK|UNK|UNK प्रचंड|UNK|UNK|UNK निसिचर|UNK|UNK|UNK समर्पित खल|UNK|UNK|UNK 3.8 बल|UNK|UNK|UNK हने|UNK|UNK|UNK नित्यं अवतार|UNK|UNK|UNK नर|UNK|UNK|UNK ध्येय ध्यानचंद बिभंजि|UNK|UNK|UNK दारुन|UNK|UNK|UNK दुख|UNK|UNK|UNK दहे|UNK|UNK|UNK इनका प्रनतपाल|UNK|UNK|UNK देव साहब संबोधित संजुक्त|UNK|UNK|UNK सक्ति|UNK|UNK|UNK नमामहे|UNK|UNK|UNK नित्यं नित्यं 1|UNK|UNK|UNK [1111111111111111111111111111111111111]  [total=-2418.329] core=(-2200.000,-37.000,37.000,-69.903,-54.405,-7.000,-471.533)  
BEST TRANSLATION: प्यारे बिषम|UNK|UNK|UNK बस|UNK|UNK|UNK नाग|UNK|UNK|UNK कन्याएं सुरासुर|UNK|UNK|UNK नर|UNK|UNK|UNK फूल बनोगे इसलिये अग|UNK|UNK|UNK जग|UNK|UNK|UNK पंथ|UNK|UNK|UNK भ्रमत|UNK|UNK|UNK अमित|UNK|UNK|UNK निसि|UNK|UNK|UNK दिवस|UNK|UNK|UNK काल करं शान्तं गुननि|UNK|UNK|UNK भरे|UNK|UNK|UNK प्रोफेसर कृष्ण नेकहा करु